In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns
import statsmodels.api as sm
from functools import reduce
import scipy.stats as stats
from collections import OrderedDict
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import random

# suppress warning

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
# display all columns
# avoid scientific digit
pd.options.display.float_format = '{:.8f}'.format

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
new_df = pd.read_csv('/content/drive/My Drive/X999/new_df.csv')

In [ ]:
new_df['building_age'] = 2024 - new_df['yearbuilt']

In [ ]:
identifier_columns = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number',
 'executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year','month_year']
metrics_columns = ['eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']
building_columns = ['numfloors', 'unitsres', 'bldgarea', 'bldgclass','building_type', 'architectural_style', "building_age",
                    'building_category', 'is_condo','is_llc']
complaints_columns = ['air_quality', 'animal_issues', 'appliances',  'building_exterior', 'doors_windows', 'electrical_issues', 'elevator_issues',
'floors_stairs','general_complaints', 'graffiti_posting', 'heat_hot_water','homeless_issues', 'noise_complaints','other_issues', 'pest_issues',
'plumbing_issues', 'police_matters', 'public_nuisance','safety_concerns', 'sanitation_issues', 'walls_ceilings', 'total_complaints']
other_features_columns = ['avg_idr','avg_idr_change','income_change','income_change_ratio',
'median_income', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4', 'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp',
 'ep_uninsur', 'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh', 'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp', 'ep_asian',
 'ep_nhpi', 'ep_twomore', 'ep_otherrace', 'ep_minrty', 'ep_white']
# new_df['avg_change_abs']

In [ ]:
# idr_evi.columns.to_list()
all_columns = identifier_columns + metrics_columns + building_columns + complaints_columns + other_features_columns
len(all_columns)

84

In [ ]:
feature_columns = building_columns + complaints_columns + other_features_columns
len(feature_columns)

61

In [ ]:
new_df = new_df[all_columns]
new_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,89,0,4,84.00000000,90,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,97,1,4,74.00000000,90,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,89,1,94,58.00000000,90,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
3,Lower East Side,0301867/23_6498,1003820001,0301867/23,6498,1 HAVEN PLAZA,27C,2023-06-20,MANHATTAN,10009,40.72701600,-73.97644000,3.00000000,2.00000000,28.00000000,1083141,Lower East Side,2023,2023-06,

In [ ]:
new_df[feature_columns].head()

,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,6.00000000,813.00000000,513000.00000000,89,0,4,84.00000000,90,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000
1,14.00000000,1861.00000000,1658649.00000000,97,1,4,74.00000000,90,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
2,26.00000000,371.00000000,453600.00000000,89,1,94,58.00000000,90,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
3,26.00000000,371.00000000,453600.00000000,89,1,94,58.00000000,90,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
4,5.00000000,20.00000000,9775.00000000,5,0,23,124.00000000,68,0,0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,40.00000000,0.00000000,11.00000000,0.00000000,2.00000000,4.00000000,0.00000000,0.00000000,0.00000000,1.00000000,0.00000000,58.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000

In [ ]:
new_df.bldgclass.unique()

array([89, 97,  5, 92, 71, 17, 18, 45, 98, 52, 84, 82, 58, 10, 21, 14, 87,
       74, 37,  4, 22, 11, 64, 13, 95,  8, 46, 88, 75, 93, 60, 55, 51, 16,
       25, 69])

In [ ]:
new_df.building_category.unique()

array([90, 68, 18, 10, 20, 16])

In [ ]:
unique_strings1 = new_df['bldgclass'].unique()
unique_strings2 = new_df['building_category'].unique()
unique_strings3 = new_df['architectural_style'].unique()
# unique_strings4 = new_df['decade'].unique()

string_to_label1 = {string: random.randint(1, 100) for string in unique_strings1}
string_to_label2 = {string: random.randint(1, 100) for string in unique_strings2}
string_to_label3 = {string: random.randint(1, 100) for string in unique_strings3}
# string_to_label4 = {string: random.randint(1, 100) for string in unique_strings4}

new_df['bldgclass_label'] = new_df['bldgclass'].map(string_to_label1)
new_df['building_category_label'] = new_df['building_category'].map(string_to_label2)
new_df['architectural_style_label'] = new_df['architectural_style'].map(string_to_label3)
# new_df['decade_label'] = new_df['decade'].map(string_to_label)

In [ ]:
# new_df.age.unique()

In [ ]:
# string_to_label4 = {'1940-1949': 1940, '1950-1959':1950, '1960-1969':1960, '1900-1909':1900, '1910-1919':1910, '1980-1989':1980, '1970-1979':1970, '1930-1939':1930, '1920-1929':1920, '1990-1999':1990, '2000-2009':2000, 'Pre-1900':1890, '2010-2020':2010}
# new_df['decade_label'] = new_df['decade'].map(string_to_label4)

In [ ]:
# new_df.building_age.unique()

In [ ]:
new_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass_label,building_category_label,architectural_style_label
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,89,0,4,84.00000000,90,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,27,74,71
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,97,1,4,74.00000000,90,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,19,74,71
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,89,1,94,58.00000000,90,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,27,74,56
3,Lower East Side,0301867/23_6498,1003820001,0301867/23,6498,1 HAVEN PLAZA,27C,2023-06-20,MANHATTAN,10009,40.

In [ ]:
new_df.drop(columns=['bldgclass', 'building_category', 'architectural_style'], inplace=True)

In [ ]:
new_df.rename(columns={'bldgclass_label': 'bldgclass', 'building_category_label': 'building_category', \
                       'architectural_style_label': 'architectural_style'}, inplace=True)

In [ ]:
new_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,27,74,71
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,1,74.00000000,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,19,74,71
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,1,58.00000000,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,27,74,56
3,Lower East Side,0301867/23_6498,1003820001,0301867/23,6498,1 HAVEN PLAZA,27C,2023-06-20,MANHATTAN,10009,40.72701600,-73.97644000,3.00000000,2.00000000,28.00000000,1083141,Lower East Side,2023,2023-0

In [ ]:
new_df.columns

Index(['Location', 'primary_key', 'bbl', 'court_index_number', 'docket_number',
       'eviction_address', 'eviction_apartment_number', 'executed_date',
       'borough', 'zipcode', 'latitude', 'longitude', 'community_board',
       'council_district', 'census_tract', 'bin', 'nta', 'year', 'month_year',
       'eviction_count_nta', 'eviction_count_building', 'eviction_count_unit',
       'eviction_count_zipcode', 'numfloors', 'unitsres', 'bldgarea',
       'building_type', 'building_age', 'is_condo', 'is_llc', 'air_quality',
       'animal_issues', 'appliances', 'building_exterior', 'doors_windows',
       'electrical_issues', 'elevator_issues', 'floors_stairs',
       'general_complaints', 'graffiti_posting', 'heat_hot_water',
       'homeless_issues', 'noise_complaints', 'other_issues', 'pest_issues',
       'plumbing_issues', 'police_matters', 'public_nuisance',
       'safety_concerns', 'sanitation_issues', 'walls_ceilings',
       'total_complaints', 'avg_idr', 'avg_idr_change', '

In [ ]:
new_df.to_csv('/content/drive/My Drive/X999/new_df_unit.csv', index=False)

## **nta_df normalized**

In [ ]:
scaler = StandardScaler()

In [ ]:
unit_df_normalized = new_df.copy()
unit_df_normalized = scaler.fit_transform(unit_df_normalized[complaints_columns + building_columns + other_features_columns])
unit_df_normalized = pd.DataFrame(unit_df_normalized, columns=complaints_columns + building_columns + other_features_columns)

In [ ]:
unit_df_normalized.head(1)

,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,-0.06971794,0.81877905,0.36540890,-1.36362253,-0.71188874,-0.16934070,0.00370864,0.16687340,-0.25457296,-0.86331212,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230


In [ ]:
identifier_columns = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number',
 'executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year','month_year']
metrics_columns = ['eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']

In [ ]:
unit_df_normalized[['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                      'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']] = new_df[['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                      'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']]
unit_df_normalized.head()

,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode
0,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,-0.06971794,0.81877905,0.36540890,-1.36362253,-0.71188874,-0.16934070,0.00370864,0.16687340,-0.25457296,-0.86331212,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000
1,-0.21785384,2.46930182,-0.24441651,-0.23014407,-0.23378550,-0.41575982,0.10719624,-0.45963993,-0.36198386,-0.20618317,-0.28308591,2.03610482,1.79042197,2.86113171,0.36455922,-0.23116852,0.09503932,3.24885496,-0.23214575,0.39008113,-0.41691376,1.11511494,1.21444407,2.18244007,1.64893560,-1.66156991,1.40471388,-0.16934070,-0.32998824,0.16687340,-0.25457296,-0.86331212,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000
2,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0.60546195,0.38978462,-0.06594408,0.17064092,-0.20618317,0.08827574,-0.09481040,-0.02177249,-0.18561793,-0.00450751,-0.01542219,-0.07560289,0.17906456,-0.02684016,0.03419009,-0.14815433,0.04264942,3.14068707,0.24364720,0.29886017,-1.36362253,1.40471388,-0.97173209,-0.86390326,0.16687340,-0.25457296,-0.86331212,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000
3,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.1184

In [ ]:
# last_column = nta_df_normalized.pop('nta')
# nta_df_normalized.insert(0, 'nta', last_column)
# nta_df_normalized.head()

In [ ]:
columns_to_move = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                      'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']

In [ ]:
for col in columns_to_move:
    last_column = unit_df_normalized.pop(col)
    unit_df_normalized.insert(0, col, last_column)
unit_df_normalized.head()

,eviction_count_zipcode,eviction_count_unit,eviction_count_building,eviction_count_nta,month_year,year,nta,bin,census_tract,council_district,community_board,longitude,latitude,zipcode,borough,executed_date,eviction_apartment_number,eviction_address,docket_number,court_index_number,bbl,primary_key,Location,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,61.00000000,0.00073801,0.60000000,56.72749658,2019-03,2019,Lower East Side,1077523,202.00000000,2.00000000,3.00000000,-73.98286600,40.71138100,10002,MANHATTAN,2019-03-15,05E,630 WATER STREET,115084,014679/18,1002600001,014679/18_115084,Lower East Side,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,-0.06971794,0.81877905,0.36540890,-1.36362253,-0.71188874,-0.16934070,0.00370864,0.16687340,-0.25457296,-0.86331212,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230
1,67.00000000,0.00042988,0.80000000,56.72749658,2018-08,2018,Lower East Side,1078059,20.00000000,2.00000000,3.00000000,-73.97489500,40.72071700,10009,MANHATTAN,2018-08-22,11E,711 FDR DR,112423,015506/17,1003560001,015506/17_112423,Lower East Side,-0.21785384,2.46930182,-0.24441651,-0.23014407,-0.23378550,-0.41575982,0.10719624,-0.45963993,-0.36198386,-0.20618317,-0.28308591,2.03610482,1.79042197,2.86113171,0.36455922,-0.23116852,0.09503932,3.24885496,-0.23214575,0.39008113,-0.41691376,1.11511494,1.21444407,2.18244007,1.64893560,-1.66156991,1.40471388,-0.16934070,-0.32998824,0.16687340,-0.25457296,-0.86331212,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
2,67.00000000,0.00215633,0.80000000,56.72749658,2023-05,2023,Lower East Side,1083144,28.00000000,2.00000000,3.00000000,-73.97598600,40.72682400,10009,MANHATTAN,2023-05-26,15D,3 HAVEN PLAZA,5780,0300290/22,1003820001,0300290/22_5780,Lower East Side,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0.60546195,0.38978462,-0.06594408,0.17064092,-0.20618317,0.08827574,-0.09481040,-0.02177249,-0.18561793,-0.00450751,-0.01542219,-0.07560289,0.17906456,-0.02684016,0.03419009,-0.14815433,0.04264942,3.14068707,0.24364720,0.29886017,-1.36362253,1.40471388,-0.97173209,-0.86390326,0.16687340,-0.25457296,-0.86331212,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
3,67.00000000,0.00215633,0.80000000,56.72749658,2023-06

In [ ]:
last_column = unit_df_normalized.pop('primary_key')
unit_df_normalized.insert(0, 'primary_key', last_column)
unit_df_normalized.head()

,primary_key,eviction_count_zipcode,eviction_count_unit,eviction_count_building,eviction_count_nta,month_year,year,nta,bin,census_tract,council_district,community_board,longitude,latitude,zipcode,borough,executed_date,eviction_apartment_number,eviction_address,docket_number,court_index_number,bbl,Location,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,014679/18_115084,61.00000000,0.00073801,0.60000000,56.72749658,2019-03,2019,Lower East Side,1077523,202.00000000,2.00000000,3.00000000,-73.98286600,40.71138100,10002,MANHATTAN,2019-03-15,05E,630 WATER STREET,115084,014679/18,1002600001,Lower East Side,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,-0.06971794,0.81877905,0.36540890,-1.36362253,-0.71188874,-0.16934070,0.00370864,0.16687340,-0.25457296,-0.86331212,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230
1,015506/17_112423,67.00000000,0.00042988,0.80000000,56.72749658,2018-08,2018,Lower East Side,1078059,20.00000000,2.00000000,3.00000000,-73.97489500,40.72071700,10009,MANHATTAN,2018-08-22,11E,711 FDR DR,112423,015506/17,1003560001,Lower East Side,-0.21785384,2.46930182,-0.24441651,-0.23014407,-0.23378550,-0.41575982,0.10719624,-0.45963993,-0.36198386,-0.20618317,-0.28308591,2.03610482,1.79042197,2.86113171,0.36455922,-0.23116852,0.09503932,3.24885496,-0.23214575,0.39008113,-0.41691376,1.11511494,1.21444407,2.18244007,1.64893560,-1.66156991,1.40471388,-0.16934070,-0.32998824,0.16687340,-0.25457296,-0.86331212,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
2,0300290/22_5780,67.00000000,0.00215633,0.80000000,56.72749658,2023-05,2023,Lower East Side,1083144,28.00000000,2.00000000,3.00000000,-73.97598600,40.72682400,10009,MANHATTAN,2023-05-26,15D,3 HAVEN PLAZA,5780,0300290/22,1003820001,Lower East Side,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0.60546195,0.38978462,-0.06594408,0.17064092,-0.20618317,0.08827574,-0.09481040,-0.02177249,-0.18561793,-0.00450751,-0.01542219,-0.07560289,0.17906456,-0.02684016,0.03419009,-0.14815433,0.04264942,3.14068707,0.24364720,0.29886017,-1.36362253,1.40471388,-0.97173209,-0.86390326,0.16687340,-0.25457296,-0.86331212,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
3,0301867/23_6498,67.00000000,0.00215633,0.80000000,56.

In [ ]:
# last_columns = nta_df_normalized.pop('Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','nta','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
#                       'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_nta')
# nta_df_normalized.insert(1, last_columns)
# nta_df_normalized.head()

In [ ]:
unit_df_normalized.to_csv('/content/drive/My Drive/X999/unit_df_normalized.csv', index=False)

In [ ]:
# type(nta_df_normalized['eviction_count_nta'] = np

In [ ]:
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_unit + 1) ~ avg_idr + avg_idr_change + ep_afam + income_change + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150 + rpl_themes + unitsres + building_age + total_complaints",
    data=unit_df_normalized
).fit()
print(final_model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     np.log(eviction_count_unit + 1)   R-squared:                       0.163
Model:                                         OLS   Adj. R-squared:                  0.163
Method:                              Least Squares   F-statistic:                     996.6
Date:                             Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                     19:21:40   Log-Likelihood:                 88709.
No. Observations:                            66395   AIC:                        -1.774e+05
Df Residuals:                                66381   BIC:                        -1.773e+05
Df Model:                                       13                                         
Covariance Type:                         nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.

In [ ]:
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_unit + 1) ~ avg_idr + avg_idr_change + ep_afam + income_change + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150 + rpl_themes + unitsres + building_age + total_complaints",
    data=new_df
).fit()
print(final_model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     np.log(eviction_count_unit + 1)   R-squared:                       0.163
Model:                                         OLS   Adj. R-squared:                  0.163
Method:                              Least Squares   F-statistic:                     996.6
Date:                             Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                     19:21:40   Log-Likelihood:                 88709.
No. Observations:                            66395   AIC:                        -1.774e+05
Df Residuals:                                66381   BIC:                        -1.773e+05
Df Model:                                       13                                         
Covariance Type:                         nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.